# The shallow water equations

In this notebook, we'll move to a more interesting problem: the shallow water equations.
The shallow water equations are one of the simplest mathematical models in geophysical fluid dynamics.
Nonetheless, they exhibit a very large share of the complexities of more realistic physics models like the Boussinesq equations.
They are a much more powerful test for the quality of spatial and temporal discretization schemes than the scalar advection equation.

The most basic form of the shallow water system describes the evolution of the thickness $h$ and velocity $u$ of an incompressible fluid flowing over a bed with elevation $b$.
They are:

$$\begin{align}
\partial_th + \nabla\cdot hu & = 0 \\
\partial_thu + \nabla\cdot\left\{hu\otimes u + \frac{1}{2}gh^2I\right\} & = -gh\nabla b
\end{align}$$

where $g$ is the acceleration due to gravity and $I$ is the 2D identity matrix.
The divergence in the second equation should be thought of as the divergence of a tensor field.
The symbol "$\otimes$" denotes the *outer product*.
You can think of the outer product of two vectors as the matrix $u\cdot u^\top$, where $\top$ denotes the transpose.
Alternatively, if you like indices, the components of the outer product are $(u \otimes u)_{ij} = u_iu_j$.

This form of the shallow water equations has an aggravating difficulty in that the time derivative is really of the *momentum*.
In this form of the problem, the momentum is a non-trivial derived function of the thickness and velocity.
It's possible to design around this when coding up numerical solvers but it is extremely annoying and as a consequence we'll instead work with the thickness and the momentum

$$q = hu$$

to arrive at the system

$$\begin{align}
\partial_th + \nabla\cdot q & = 0 \\
\partial_tq + \nabla\cdot\left\{h^{-1}q\otimes q + \frac{1}{2}gh^2I\right\} & = -gh\nabla b.
\end{align}$$

In this equivalent form of the problem, the time derivative is on just the momentum and not some function of the momentum and other variables.
We can then easily implement common timestepping schemes.

### Problem setup

First, we'll use a periodic rectangle as our spatial domain of 20m to a side -- a little less than the width of an Olympic swimming pool.
The periodicity is a little artificial and in later demos we'll show how to add realistic boundary conditions.

In [ ]:
import firedrake
nx, ny = 32, 32
Lx, Ly = 20., 20.
mesh = firedrake.PeriodicRectangleMesh(
    nx, ny, Lx, Ly, diagonal='crossed'
)

As an initial condition, we'll take the water to be 1m deep, and to perturb this a bit we'll add a 10cm parabolic burp near one corner.

In [ ]:
from firedrake import inner, max_value, Constant
x = firedrake.SpatialCoordinate(mesh)
lx = 5.
y = Constant((lx, lx))
r = Constant(2.5)

H = Constant(1.)
δh = Constant(0.1)
h_expr = H + δh * max_value(0, 1 - inner(x - y, x - y) / r**2)

To make things yet more interesting, we'll add some variable bottom topography consisting of another parabolic burp.

In [ ]:
y = Constant((3 * lx, 3 * lx))
δb = Constant(1/4)
b = δb * max_value(0, 1 - inner(x - y, x - y) / r**2)

This is all the input data we need.

### DG(1) discretization

In the last demo, we saw that using a higher-resolution scheme than DG(0) gave a huge improvement in the accuracy of the solution.
Here we'll do the same thing and start with a DG(1) discretization for both thickness and momentum.
For the momentum, we'll create a vector rather than a scalar function space.

In [ ]:
degree = 1
Q = firedrake.FunctionSpace(mesh, family='DG', degree=degree)
V = firedrake.VectorFunctionSpace(mesh, family='DG', degree=degree)

As we saw in the last demo, the solvers are expecting to work with one single monolithic state vector.
What we want is the cartesian product $Z = Q \times V$ of the thickness space $Q$ and the momentum space $V$, and Firedrake has some built-in notation for doing just that.

In [ ]:
Z = Q * V
z_0 = firedrake.Function(Z)

In order to initialize the state vector $z_0$, we can split out the thickness and momentum components and project the expression we wrote above.

In [ ]:
h_0, q_0 = z_0.split()
h_0.project(h_expr - b);

Now we'll do just like we did before: create a function that will return the right-hand side of the shallow water equations and then create a numerical solver for those equations.

In [ ]:
from plumes import models
from plumes.coefficients import gravity
g = firedrake.Constant(gravity)
equation = models.shallow_water.make_equation(g, b)

The wave speed for the shallow water equations is $\sqrt{g\cdot h}$, which gives us a speed to guess at a CFL-stable timestep with.

In [ ]:
import numpy as np
C = np.sqrt(gravity * (float(H) + float(δh)))
δx = mesh.cell_sizes.dat.data_ro[:].min()
timestep = (δx / 8) / C / (2 * degree + 1)
print(timestep)

We can also use this to guess how long it will take for a wave to propagate all the way across the domain.

In [ ]:
final_time = 4 * Lx / C
num_steps = int(final_time / timestep)
print(f'final time: {final_time:5.3f}')
print(f'num steps:  {num_steps}')
dt = final_time / num_steps

We'll want to make movies again, so we need to calculate how the output frequency.

In [ ]:
output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)
print(f'output frequency: {output_freq}')

Now we can make our solver and proceed like before.

In [ ]:
from plumes import numerics
integrator = numerics.ExplicitEuler(equation, z_0, dt)

In the timestepping loop, we'll add a bit of diagnostic information to the progress bar to make sure the simulation hasn't exploded.

In [ ]:
import tqdm

hs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h = z.split()[0]
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
    
    integrator.step(dt)

Movie time!

In [ ]:
Q0 = firedrake.FunctionSpace(mesh, family='DG', degree=0)
η = firedrake.project(hs[0] + b, Q0)

In [ ]:
%%capture
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
colors = firedrake.tripcolor(
    η, num_sample_points=1, vmin=0.75, vmax=1.25, axes=axes
)
fig.colorbar(colors)

from matplotlib.animation import FuncAnimation
def animate(h):
    η.project(h + b)
    colors.set_array(η.dat.data_ro[:])

interval = 1e3 * output_freq * dt
animation = FuncAnimation(fig, animate, frames=hs, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())

### BDFM discretization

Now we'll make a better choice of finite element space.

In [ ]:
degree = 2
V = firedrake.FunctionSpace(mesh, family='BDFM', degree=degree)

Z = Q * V
z_0 = firedrake.Function(Z)

h_0, q_0 = z_0.split()
h_0.project(h_expr - b);

In [ ]:
timestep = (δx / 12) / C / (2 * degree + 1)
num_steps = int(final_time / timestep)
print(f'final time: {final_time:5.3f}')
print(f'num steps:  {num_steps}')
dt = final_time / num_steps

output_time = 1 / 30
output_freq = max(int(output_time / dt), 1)
print(f'output frequency: {output_freq}')

In [ ]:
integrator = numerics.ExplicitEuler(equation, z_0, dt)

In [ ]:
hs = []

progress_bar = tqdm.trange(num_steps)
for step in progress_bar:
    if step % output_freq == 0:
        z = integrator.state
        h = z.split()[0]
        hmin, hmax = h.dat.data_ro[:].min(), h.dat.data_ro[:].max()
        progress_bar.set_description(f'{hmin:5.3f}, {hmax:5.3f}')
        hs.append(h.copy(deepcopy=True))
    
    integrator.step(dt)

In [ ]:
η = firedrake.project(hs[0] + b, Q0)

In [ ]:
%%capture
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
axes.get_xaxis().set_visible(False)
axes.get_yaxis().set_visible(False)
colors = firedrake.tripcolor(
    η, num_sample_points=1, vmin=0.75, vmax=1.25, axes=axes
)
fig.colorbar(colors)

from matplotlib.animation import FuncAnimation
def animate(h):
    η.project(h + b)
    colors.set_array(η.dat.data_ro[:])

interval = 1e3 * output_freq * dt
animation = FuncAnimation(fig, animate, frames=hs, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_html5_video())